<a href="https://colab.research.google.com/github/ElizavetaNosova/HSE_ML_homework/blob/master/NosovaML_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
categories = ['sci.crypt', 'sci.med', 'comp.graphics', 'talk.politics.guns']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
vectorizer = sklearn.feature_extraction.text.CountVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)

In [0]:
import pandas as pd


In [0]:
from sklearn.model_selection import KFold, train_test_split
from statistics import mean
X = vectors
y = newsgroups_train.target
X, X_test, y, y_test = train_test_split(X, y, random_state=42)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

def cross_validation(model):
    valid_scores = []
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        model.fit(X_train, y_train)    
        y_pred_test = model.predict(X_test)
        y_pred_valid = model.predict(X_valid)
        valid_score = sklearn.metrics.f1_score(y_valid, y_pred_valid, average='macro')
        valid_scores.append(valid_score)
    test_score = sklearn.metrics.f1_score(y_test, y_pred_test, average='macro')
    print('Cross Validation mean score:', mean(valid_scores))
    print("Score on test data: {0:.4f}".format(test_score))

array([3, 2, 0, ..., 1, 2, 2])

In [0]:
parameters_LogisticRegression = {'class_weight' : ['balanced', None],
                  'C' : [0.01, 0.1, 1.0, 10.0],
                  'max_iter': [500, 1000]
                 }

parameters_RandomForest = {'n_estimators': [5, 10, 15],
                           'max_depth': [None, 5],
                           'min_samples_split': [0.5, 1.0, 5]                          
                }

parameters_kNeigbors = {'n_neighbors': [3, 5, 10],
                        'p': [1,2]
                }

In [0]:
from sklearn.model_selection import GridSearchCV
def grid_search(model, parameters):
    grid_search = GridSearchCV(model, param_grid=parameters, cv=folds, scoring='f1_macro')
    grid_search.fit(X, y)
    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))

In [36]:
from sklearn import linear_model
grid_search(linear_model.LogisticRegression(), parameters_LogisticRegression)
#Я случайно перезапустила ячейку, найденные параметры выписаны ниже

KeyboardInterrupt: ignored

In [34]:
from sklearn import ensemble
grid_search(ensemble.RandomForestClassifier(), parameters_RandomForest)

Best score: 0.8604137830119477
Best parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 15}


In [35]:
grid_search(sklearn.neighbors.KNeighborsClassifier(), parameters_kNeigbors)

Best score: 0.6943276662309239
Best parameters: {'n_neighbors': 3, 'p': 2}


In [0]:
chosenLogisticRegression = linear_model.LogisticRegression(C=1.0, class_weight=None, max_iter=500)
chosenRandomForest = ensemble.RandomForestClassifier(max_depth=None, min_samples_split=5, n_estimators=15)
chosenKNeigbors = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 3, p = 2)

In [0]:
cross_validation(chosenLogisticRegression)

Cross Validation mean score: 0.938068071731182
Score on test data: 0.9501


In [0]:
cross_validation(chosenRandomForest)

Cross Validation mean score: 0.8651362356906477
Score on test data: 0.8595


In [0]:
cross_validation(chosenKNeigbors)

Cross Validation mean score: 0.694327666230924
Score on test data: 0.7040


Результаты предсказаний на тесте могут отличаться от результатов кросс-валдации как в большую, так и в меньшую сторону, но не существенно (в пределах 1,5 процентов). На какое бы число мы ни ориентировались, лучшие результаты даёт логистическая регрессия, худшие - K ближайших соседей.

In [0]:
import eli5  
import pandas as pd  

In [0]:
LRmodel = chosenLogisticRegression.fit(X, y)
RFmodel = chosenRandomForest.fit(X,y)
KNmodel = chosenKNeigbors.fit(X,y)

In [0]:
index_to_word = {v:k for k,v in vectorizer.vocabulary_.items()}

In [0]:
df = eli5.formatters.as_dataframe.explain_weights_df(LRmodel)


In [0]:
def analyze_features(model, n=10):
    df = eli5.formatters.as_dataframe.explain_weights_df(model)
    for i in range(len(categories)):
        category_df = df[df['target']==i]
        print('КАТЕГОРИЯ', i)
        category_df = category_df.sort_values(['weight'], ascending=False)
        ids = list(category_df['feature'])
        printed = 0
        for i in ids:
            if printed == n:
                break
            else:
                try:  #как минимум индекс BIAS возможен в списке, но его нет в словаре
                    key = int(i[1:])
                    print(index_to_word[key])
                    printed += 1
                except:
                    pass

In [0]:
analyze_features(LRmodel, 10)

КАТЕГОРИЯ 0
graphics
image
3d
files
images
looking
software
points
file
card
КАТЕГОРИЯ 1
clipper
key
encryption
chip
gtoal
security
pgp
nsa
code
keys
КАТЕГОРИЯ 2
doctor
msg
pitt
my
disease
health
photography
treatment
dyer
information
КАТЕГОРИЯ 3
gun
guns
waco
batf
atf
usa
firearms
cathy
rights
them


В модели логистической регрессии подборки слов позволяют однозначно узнать изначальную тему).
Видимо, номера тем не совпадают с порядком, в котором я их указывала: медицина была второй (первой, считая от нуля), а стала предпоследней темой.
Есть слова, отнесённые не в ту группу (photography попало в медицину). С другими категориями никаких проблем нет. Слово them попало в категорию оружия, вероятно, в связи с тем, что в текстах этой категории чаще всего совершают действия, направленные на несколько человек (в остальных случаях объект чаще неодушевлённый или - в случае с медициной - 1 человек)

In [0]:
##Повторим обучение, оставив только 500 самых важных слов
df = eli5.formatters.as_dataframe.explain_weights_df(LRmodel)
df = df.sort_values(['weight'], ascending=False)
important_featuresLR = list(df.feature)[:500]
important_wordsLR = []
for i in important_featuresLR:
    try:
        important_wordsLR.append(index_to_word[int(i[1:])])
    except:
        pass
print(important_wordsLR)
            

['gun', 'clipper', 'graphics', 'key', 'guns', 'encryption', 'chip', 'waco', 'gtoal', 'security', 'doctor', 'pgp', 'msg', 'image', 'pitt', 'batf', 'my', '3d', 'atf', 'files', 'usa', 'disease', 'firearms', 'images', 'looking', 'nsa', 'code', 'software', 'points', 'keys', 'file', 'card', 'crypto', 'des', 'health', 'package', 'photography', 'treatment', 'once', 'cathy', 'dyer', 'information', 'cview', 'tiff', '3do', 'windows', 'gif', 'mail', 'cryptography', 'na', 'hi', 'banks', 'people', 'secret', 'tapped', 'polygon', 'netcom', 'your', 'radford', 'rights', '42', 'she', 'advance', '24', 'them', 'geb', 'effects', 'feustel', 'program', 'hp', 'smith', 'rice', 'computer', 'us', 'texas', 'format', 'krillean', 'access', 'should', 'eff', 'tx', 'sgi', 'fbi', 'public', 're', 'marc', 'vesa', 'color', 'post', 'vga', 'jeeves', 'missouri', 'pollux', 'book', 'politics', 'need', 'conference', 'gordon', 'video', 'tempest', 'ranch', 'medical', 'eye', 'nagle', 'animation', 'more', 'wiretap', 'au', 'research'

In [0]:
import re
def pseudotext(words, text):
    tokens = [token for token in re.findall('[a-zA-Z]+', text) if token in words]
    pseudotext = ' '.join(tokens)
    return pseudotext

In [0]:
important_words_data = [pseudotext(important_wordsLR, text) for text in newsgroups_train.data]

In [0]:
X = vectorizer.transform(important_words_data)
y = newsgroups_train.target
X, X_test, y, y_test = train_test_split(X, y, random_state=42)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

In [0]:
grid_search(linear_model.LogisticRegression(), parameters_LogisticRegression)

Best score: 0.933409456986543
Best parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 500}


In [0]:
cross_validation(linear_model.LogisticRegression(C=1.0, class_weight='balanced', max_iter=500))

Cross Validation mean score: 0.933409456986543
Score on test data: 0.9032


Переобучение стало проявляться сильнее (больше разница между валидацией и тестом), качество работы ухудшилось. 
Проверим, не связано ли это с тем, что стало много пустых псевдотекстов, которые можно распределить по категориям только наугад

In [0]:
len([text for text in important_words_data if not text])

2

Не связано.

Случайный лес. Метод feature_importances_

In [0]:
RFmodel.feature_importances_

array([0.00012498, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

С помощью функции feature_importances_, не получается понять, к какому решению склоняет какое слово.
Посмотрим на слова с ненулевым значением
Если они будут похожи на соответствующие выбранным темам, натренируем модель с теми же параметрами на выявление каждой темы по отдельности, чтобы посмотреть списки слов по тематическим категориям

In [0]:
importances = RFmodel.feature_importances_
not_zero = [i for i in importances if i]
print(len(not_zero))

3198


Ненулевых элементов достаточно много.
Посмотрим, сколько элементов вносит не менее 1.5 от среднего значения и какие это слова

In [0]:
import statistics
mean = statistics.mean(not_zero)
print(len([i for i in not_zero if i > 1.5*mean]))

440


In [0]:
important_wordsRT = []
for i in range(len(importances)):
    if importances[i] > 1.5*mean:
        important_wordsRT.append(index_to_word[i])

In [0]:
important_wordsRT

['15',
 '24',
 '256',
 '3d',
 '54',
 '7dv',
 '80',
 'able',
 'about',
 'access',
 'agencies',
 'agents',
 'agree',
 'algorithm',
 'all',
 'allergic',
 'am',
 'amendment',
 'an',
 'and',
 'anecdotal',
 'animation',
 'announcement',
 'any',
 'appears',
 'appreciated',
 'are',
 'argue',
 'as',
 'atf',
 'attacks',
 'auto',
 'available',
 'banks',
 'batf',
 'be',
 'been',
 'believe',
 'bellovin',
 'between',
 'bill',
 'body',
 'boi',
 'bontchev',
 'book',
 'both',
 'branch',
 'building',
 'burns',
 'but',
 'c5rpoj',
 'ca',
 'caltech',
 'can',
 'candida',
 'car',
 'cards',
 'case',
 'cc',
 'cdt',
 'cell',
 'chaos',
 'cheap',
 'child',
 'chip',
 'choose',
 'chopin',
 'chronic',
 'citizens',
 'claim',
 'client',
 'clinical',
 'clinton',
 'clipper',
 'code',
 'cold',
 'color',
 'colorado',
 'com',
 'communications',
 'computer',
 'concealed',
 'cont',
 'control',
 'conversations',
 'court',
 'crime',
 'criminal',
 'criminals',
 'crypt',
 'crypto',
 'cs',
 'cwru',
 'data',
 'daughter',
 'day',
 

Список какой-то странный получился: много слов не связанных с категориями (but, допустим). Числа нам не мешают: мы их не включаем в регулярку, когда пересобираем псевдотексты для новых векторов.

In [0]:
important_words_dataRT = [pseudotext(important_wordsRT, text) for text in newsgroups_train.data]

In [0]:
X = vectorizer.transform(important_words_dataRT)
y = newsgroups_train.target
X, X_test, y, y_test = train_test_split(X, y, random_state=42)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

In [0]:
grid_search(ensemble.RandomForestClassifier(), parameters_RandomForest)

Best score: 0.7998041420079826
Best parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 15}


In [0]:
cross_validation(chosenRandomForest)

TypeError: ignored

Кажется, надо изменить функцию

In [0]:
def cross_validation(model):
    valid_scores = []
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        model.fit(X_train, y_train)    
        y_pred_test = model.predict(X_test)
        y_pred_valid = model.predict(X_valid)
        valid_score = sklearn.metrics.f1_score(y_valid, y_pred_valid, average='macro')
        valid_scores.append(valid_score)
    test_score = sklearn.metrics.f1_score(y_test, y_pred_test, average='macro')
    print('Cross Validation mean score:', sum(valid_scores)/len(valid_scores))
    print("Score on test data: {0:.4f}".format(test_score))

In [0]:
cross_validation(ensemble.RandomForestClassifier(min_samples_split=5, n_estimators=15))

Cross Validation mean score: 0.8038521701660706
Score on test data: 0.8151


Качество ухудшилось. Попробуем использовать главные слова, выбранные для модели логистической регрессии

In [0]:
X = vectorizer.transform(important_words_data)
y = newsgroups_train.target
X, X_test, y, y_test = train_test_split(X, y, random_state=42)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

grid_search(ensemble.RandomForestClassifier(), parameters_RandomForest)

Best score: 0.893520757665202
Best parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 15}


In [0]:
cross_validation(ensemble.RandomForestClassifier(min_samples_split=5, n_estimators=15))

Cross Validation mean score: 0.8927894191591517
Score on test data: 0.8561


Переобучение стало сильнее.
На тестовых данных результат сопоставим с использованием всех слов, хотя во время валидации получается более высокий результат.
Вероятно, слова с высокими весами определяют первые ветвления дерева, а фактически выбор категории зависит от слов, которые оказываются в конце цепочки.

Также не исключено, что в списке весов порядок фич не соответствовал индексам в словаре.

Чтобы проверить первую гипотезу, возьмём слова, вклад которых был сопоставим со средним (от 0.75 до 1.25)

In [0]:
important_wordsRT = []
for i in range(len(importances)):
    if importances[i] < 1.25*mean and importances[i] > 0.75*mean:
        important_wordsRT.append(index_to_word[i])

In [0]:
print(important_wordsRT)

['01', '10', '14', '1993apr18', '1qv83m', '2nd', '34aej7d', '408', '41', '4656', '56', '67', '7000ft', '93', '___', 'accused', 'action', 'acupuncturist', 'advocating', 'agency', 'aids', 'air', 'aisun3', 'al', 'algorithms', 'allocation', 'alt', 'american', 'another', 'arpa', 'art', 'article', 'assuming', 'at', 'australia', 'authorisation', 'authority', 'away', 'baby', 'barbecued', 'beethoven', 'before', 'bell', 'best', 'bit', 'bitmap', 'bms', 'boot', 'boxes', 'brinich', 'business', 'buy', 'by', 'capita', 'card', 'cares', 'cash', 'cbnews', 'cd', 'cellular', 'ch', 'checked', 'cheeses', 'chemically', 'chris', 'city', 'clear', 'colors', 'comments', 'communication', 'communist', 'compound', 'computing', 'confirm', 'constant', 'contact', 'continue', 'contracted', 'contributors', 'copro', 'correctly', 'council', 'creating', 'csd', 'cube', 'culture', 'cured', 'curves', 'cuts', 'cylindrical', 'dane', 'dave', 'decide', 'denver', 'depends', 'deuterium', 'devices', 'diagnosed', 'distant', 'dns1', '

Видно, что они не соотносятся с темами.
Наконец, посмотрим на слова, которые формируют листочки дерева

In [0]:
important_wordsRT = []
for i in range(len(importances)):
    if importances[i] < 0.75*mean and importances[i] > 0.3*mean:
        important_wordsRT.append(index_to_word[i])

In [0]:
important_wordsRT

['00',
 '00101001b',
 '001230',
 '024103',
 '0952',
 '100',
 '1000',
 '126',
 '13',
 '131239',
 '13h',
 '141',
 '150',
 '15985',
 '16',
 '163133',
 '17',
 '1787',
 '1974',
 '1993',
 '1993apr14',
 '1993apr15',
 '1993apr19',
 '1993apr21',
 '1993apr5',
 '1bfdqsj53kostz6hroshsdzlvul1',
 '1qhc2p',
 '1r3jgbinn35i',
 '203',
 '2071',
 '2073',
 '21149',
 '2117',
 '214',
 '216',
 '2273',
 '2339',
 '241',
 '250',
 '26',
 '2800',
 '2ua',
 '30',
 '3060',
 '32',
 '3576',
 '37',
 '389',
 '40',
 '4615trd',
 '4660',
 '47',
 '48',
 '488',
 '505',
 '515',
 '52',
 '5363',
 '53iss6',
 '5742',
 '6000',
 '619',
 '6300',
 '640x480',
 '68',
 '704',
 '718622',
 '7800',
 '840',
 '882',
 '8k',
 '9760',
 '__the',
 '_count',
 'aauwpiugyv2n8n',
 'aayau',
 'above',
 'acaps',
 'according',
 'acm',
 'activities',
 'acts',
 'actual',
 'actually',
 'address',
 'adept',
 'administration',
 'administrative',
 'adult',
 'advance',
 'advice',
 'after',
 'against',
 'age',
 'aggressively',
 'aj336',
 'ajiif6tsm',
 'ak949',
 '

По-прежнему выглядит так себе, но попробуем использовать их

In [0]:
important_words_dataRT = [pseudotext(important_wordsRT, text) for text in newsgroups_train.data]

In [0]:
X = vectorizer.transform(important_words_dataRT)
y = newsgroups_train.target
X, X_test, y, y_test = train_test_split(X, y, random_state=42)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

cross_validation(chosenRandomForest)

Cross Validation mean score: 0.6821735136052991
Score on test data: 0.6836


На листочках получается совсем плохо

K ближайших соседей

Воспользуемся словами, у которых были наибольшие веса в логистической регрессии

In [0]:
X = vectorizer.transform(important_words_data)
y = newsgroups_train.target
X, X_test, y, y_test = train_test_split(X, y, random_state=42)
folds = KFold(n_splits=5, shuffle=True, random_state=0)

from sklearn import neighbors
grid_search(neighbors.KNeighborsClassifier(), parameters_kNeigbors)

Best score: 0.7096129798383806
Best parameters: {'n_neighbors': 3, 'p': 2}


In [0]:
cross_validation(neighbors.KNeighborsClassifier(n_neighbors=3, p=2))

Cross Validation mean score: 0.7096129798383807
Score on test data: 0.7164


Качество немного повысилась, переобучения нет.
Методы извлечения информации о словах с самым большим весом, которые работали для логистической  регрессии и для случайного леса, для K ближайших соседей не подходят.

Подберём параметры CountVectirizer

In [0]:
def chooseCVparams(model):
    ngram_range = [(1, 1), (1, 2)]
    max_df_list = [None, 1, 2]
    min_df_list = [None, 0.5, 1]
    max_features_list = [None, 500]
    for ngram_range in ngrams:
        for max_df in max_df_list:
            for min_df in min_df_list:
                  for max_features in max_features_list:
                      try: #на случай, если есть несовместимые комбинации параметров
                            vectorizer = sklearn.feature_extraction.text.CountVectorizer(max_df=max_df, min_df=min_df,max_features=max_features)
                            X=vectorizer.fit_transform(newsgroups_train.data)
                            print(vectorizer)
                            cross_validation(model)
                        except:
                             pass

In [37]:
chooseCVparams(chosenLogisticRegression)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.938068071731182
Score on test data: 0.9501
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.938068071731182
Score on test data: 0.9501
CountVectorizer(analyzer='

Результат везде одинаковый, что странно. На других моделях (см. ниже) результат зависит от параметров векторайзера. Возможно, логистическая регрессия сама выбирает важные слова так хорошо, что настройки векторайзера не принципиально. Правда, остаётся загадкой, почему не влияют настройки биграм.

In [38]:
chooseCVparams(chosenRandomForest)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.8652434500838151
Score on test data: 0.8607
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.8627445637304301
Score on test data: 0.8843
CountVectorizer(analyzer

Лучший результат случайного леса на тесте:
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=2, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.8536516505290225
Score on test data: 0.8705

In [39]:
chooseCVparams(chosenKNeigbors)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.694327666230924
Score on test data: 0.7040
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
Cross Validation mean score: 0.694327666230924
Score on test data: 0.7040
CountVectorizer(analyzer='

Для метода K ближайших соседей настройки векторайзера тоже оказались не принципиальны.